In [1]:
import matchzoo as mz
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/home/lss9/anaconda3/envs/matchzoo/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.
/home/lss9/anaconda3/envs/matchzoo/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
train_dataset = pd.read_csv('../../train_data/train_data_1_1', delimiter=',')
validation_dataset = pd.read_csv('../../train_data/validation_data_1_1', delimiter=',')

train_dataset = train_dataset.replace(np.nan, ' ', regex=True)
validation_dataset = validation_dataset.replace(np.nan, ' ', regex=True)

In [3]:
train_dataset = train_dataset.head(1000)
validation_dataset = validation_dataset.head(1000)

In [4]:
list_data1 = []
for i, row in train_dataset.iterrows():
    
    line1 = {'id_left': str(row['article_id']),
            'text_left':str(row['article_page_title']),
            'id_right':str(row['table_id']),
            'text_right':str(row['table_page_title']),
            'label':row['label']
           }
    
    list_data1.append(line1)

df1 = pd.DataFrame(list_data1)
train_pack = mz.pack(df1)

In [5]:
list_data2 = []
for i, row in validation_dataset.iterrows():
    
    line2 = {'id_left': str(row['article_id']),
            'text_left':str(row['article_page_title']),
            'id_right':str(row['table_id']),
            'text_right':str(row['table_page_title']),
            'label':row['label']
           }
    
    list_data2.append(line2)

df2 = pd.DataFrame(list_data2)
valid_pack = mz.pack(df2)

In [6]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.MeanAveragePrecision()
]

In [20]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)

In [13]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)
valid_pack_processed = preprocessor.transform(valid_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 989/989 [00:00<00:00, 10390.52it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 996/996 [00:00<00:00, 13957.10it/s]
Building Vocabulary from a datapack.: 100%|██████████| 7734/7734 [00:00<00:00, 4096318.62it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 989/989 [00:00<00:00, 11349.26it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 996/996 [00:00<00:00, 13973.53it/s]
Processing text_right with transform: 100%|██████████| 996/996 [00:00<00:00, 172539.52it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 988/988 [00:00<00:00, 10906.97it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 992/992 [00:00<00:00, 13595.80

In [14]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f67629bac10>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f6760c1fe10>,
 'vocab_size': 3916,
 'embedding_input_dim': 3916,
 'input_shapes': [(10,), (100,)]}

In [21]:
model = mz.models.ArcII()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = 100
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [3, 3]
model.params['pool_2d_size'] = [[3, 3], [3, 3]]
model.params['optimizer'] = 'adam'
model.build()
model.compile()
model.backend.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             391600      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 10, 32)       9632        embedding[0][0]            

In [22]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [23]:
pred_x, pred_y = valid_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y), model_save_path='ARCII_title', once_every=1)

In [24]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)

In [25]:
history = model.fit_generator(train_generator, epochs=5, callbacks=[evaluate])

Epoch 1/5
1/1 [==============================] - 0s 306ms/step - loss: 1.0278
Validation: mean_average_precision(0.0): 0.5106275303643725
Epoch 2/5
1/1 [==============================] - 0s 19ms/step - loss: 0.8174
Validation: mean_average_precision(0.0): 0.5106275303643725
Epoch 3/5
1/1 [==============================] - 0s 18ms/step - loss: 0.6391
Validation: mean_average_precision(0.0): 0.5106275303643725
Epoch 4/5
1/1 [==============================] - 0s 18ms/step - loss: 0.4656
Validation: mean_average_precision(0.0): 0.5106275303643725
Epoch 5/5
1/1 [==============================] - 0s 27ms/step - loss: 0.3532
Validation: mean_average_precision(0.0): 0.5106275303643725
